# ch5 logistic回归


定义sigmoid函数

In [1]:
import numpy as np
def sigmoid(x):
    return 1.0/(1+np.exp(-x))

梯度上升法
$$
w:=w+\alpha \Delta_w f(w)
$$

其中
$$
\Delta f(x,y)=\frac{df(x,y)}{dx},\frac{df(x,y)}{dy}
$$

求f(x,y)最大值，用梯度上升，求最小值，用梯度下降，区别在于加个负号。

In [2]:
def gradAscent(traindataArr,trainlabelArr):
    # 梯度上升算法，一次迭代遍历所有数据文件
    """
    参数：训练集，标签集
    return： weights
    """
    # 转为数组类型，方便计算
    traindataArr=np.array(traindataArr)
    trainlabelArr=np.array(trainlabelArr)
    # shape为(100,)与shape为（100,1)区别：
    # 前者在矩阵运算时候可能发生广播，所以要reshape一下
    trainlabelArr=trainlabelArr.reshape(trainlabelArr.shape[0],1)
    
    # 初始化参数w，维度为(样本维数,1),即列向量
    # w=np.zeros(traindataArr.shape[1]),维度为：(1,样本维数)
    w=np.ones( (traindataArr.shape[1],1) )
    
    # 步长
    alpha=0.001
    # 迭代次数
    iter=500

    # iter次随机梯度上升
    for i in range(iter):
        # print("iter:%d /%d"%(i,iter))
        # 每一次迭代，都遍历所有样本
        
        h=sigmoid(np.dot(traindataArr,w))
        error=trainlabelArr-h
        w=w+ alpha* np.dot(traindataArr.T,error)
    # 返回weights 
    return w

In [3]:
# test

# 解析数据
def loadDataSet():
    '''
    Desc: 
        加载并解析数据
    Args:
        file_name -- 要解析的文件路径
    Returns:
        dataMat -- 原始数据的特征
        labelMat -- 原始数据的标签，也就是每条样本对应的类别。即目标向量
    '''
    # dataMat为原始数据， labelMat为原始数据的标签
    dataMat = []
    labelMat = []
    fr = open("testSet.txt")
    for line in fr.readlines():
        lineArr = line.strip().split()
        # 为了方便计算，我们将 X0 的值设为 1.0 ，也就是在每一行的开头添加一个 1.0 作为 X0
        dataMat.append([1.0, float(lineArr[0]), float(lineArr[1])])
        labelMat.append(int(lineArr[2]))
    return dataMat, labelMat
arr,label=loadDataSet()
w=gradAscent(arr,label)
w

array([[ 4.12414349],
       [ 0.48007329],
       [-0.6168482 ]])

## 5-4 随机梯度上升

梯度上升法在更新回归系数的时候，需要遍历整个数据集。

当数据集特征增多时，该方法计算复杂度也会明显增加。

随机梯度法是一次仅用一个样本来更新回归系数。由于可以在新样本之前，

对分类器进行增量式更新，所以随机梯度下降算法是一个在线学习算法。

相对的，一次处理所有数据的方法称为“批处理”方法。

随机梯度算法概述：

```
所有回归系数初始化为1
遍历每个样本：
    计算该样本梯度
    更新回归系数
返回回归系数
```

In [4]:
def randGradAscent(traindataArr,trainlabelArr):
    # 随机梯度上升，每遍历一个数据文件就更新一次参数
    """
    参数：训练集，标签集
    return： weights
    """
    # 转为数组类型，方便计算
    traindataArr=np.array(traindataArr)
    trainlabelArr=np.array(trainlabelArr)
    # shape为(100,),  取值trainlabelArr[1]=1.0
    # shape为(100,1), 取值trainlabelArr[1]=[1.0]
    # 注意类型
    trainlabelArr=trainlabelArr.reshape(trainlabelArr.shape[0],1)
    
    # 初始化参数w，维度为(样本维数,1),即列向量
    w=np.ones( (traindataArr.shape[1],1) )
    # 步长
    alpha=0.001
    # 迭代次数
    iter=500
    for i in range(iter):
        for j in range(traindataArr.shape[0]):
            alpha=4/(1.0+j+i)+0.01
            xi=traindataArr[j]
            yi=trainlabelArr[j]
            # 将(m,)reshape为(1,m)
            xi=xi.reshape(1,xi.shape[0])
            # w*xi的值，即y_hat
            h=sigmoid(np.dot(xi,w))
            # err=y-y_hat
            error=yi-h
            # 更新参数
            w = w + alpha* np.dot(xi.T,error)
    # 返回weights 
    return w

In [5]:
# test
arr,label=loadDataSet()
w=randGradAscent(arr,label)
w

array([[15.96382299],
       [ 1.28557166],
       [-2.22581784]])

## 5-5 用logistic进行分类


In [6]:
def predict(w,x):
    # 参数列表，输入数据
    # return 类别
    wx=np.dot(w.T,x)
    p=sigmoid(wx)
    if p>0.5:
        return 1.0
    else:
        return 0.0

## 5-6 modelTest

In [7]:
def loadData(filename):
    fr=open(filename)
    dataArr=[];labelArr=[]
    for line in fr.readlines():
        # 去掉多余空格，\t分割
        currLine=line.strip().split('\t')
        featList=[]
        for i in range(21):
            featList.append(float(currLine[i]))
        # 特征
        dataArr.append(featList)
        # 标签
        labelArr.append(float(currLine[21]))
    return dataArr,labelArr

In [8]:
def modelTest(testDataArr,testLabelArr,w):
    # return accur
    errCnt=0
    for i in range(len(testDataArr)):
        if testLabelArr[i]!=predict(w,testDataArr[i]):
            errCnt+=1
    # return accur
    return 1-errCnt/float(len(testLabelArr))

In [11]:
trainData, trainLabel = loadData("./horseColicTraining.txt")
testData,testLabel=loadData("./horseColicTest.txt")
# 随机梯度下降
w=randGradAscent(trainData,trainLabel)
# 梯度下降
# w=gradAscent(trainData,trainLabel)
accu=modelTest(testData,testLabel,w)
accu

0.6268656716417911

In [10]:
# a=np.ones((2,1))
a=np.ones(2)
c=a[1]
c

1.0